## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import os

import sys
sys.path.append('..')

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
data_file = os.path.join("..","Weather_Data","WeatherPy_vacation.csv")
vacation_df = pd.read_csv(data_file)

vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,44,Rikitea,PF,72.23,light rain,-23.1203,-134.9692,People ThankYou
1,56,Thompson,CA,71.76,scattered clouds,55.7435,-97.8558,Burntwood Hotel
2,65,Hami,CN,70.38,clear sky,42.8000,93.4500,Laoyutang Resort
3,69,Horadiz,AZ,71.96,clear sky,39.4488,47.3353,Horadiz Olympic Center
4,72,Moroni,KM,72.25,clear sky,-11.7022,43.2551,Hotel Les Arcades


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.

max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df["City"]=="Atikokan"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Atikokan"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Thompson"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Whitecourt"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="High Level"] 

In [6]:
vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,127,Atikokan,CA,72.14,broken clouds,48.7555,-91.6216,Quetico Inn (formerly Radisson)


In [7]:
start_np=vacation_start.to_numpy()
end_np = vacation_end.to_numpy()
stop1_np = vacation_stop1.to_numpy()
stop2_np = vacation_stop2.to_numpy()
stop3_np = vacation_stop3.to_numpy()

In [8]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = (start_np[0][5], start_np[0][6])
end = (end_np[0][5], end_np[0][6])
stop1 = (stop1_np[0][5], stop1_np[0][6])
stop2 = (stop2_np[0][5], stop2_np[0][6])
stop3 = (stop3_np[0][5], stop3_np[0][6])

In [9]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()
our_trip = gmaps.directions_layer(start, end, 
                                  waypoints=[stop1, stop2, stop3], 
                                  travel_mode='DRIVING')
fig.add_layer(our_trip)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1, vacation_stop2,
                          vacation_stop3, vacation_end], ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,127,Atikokan,CA,72.14,broken clouds,48.7555,-91.6216,Quetico Inn (formerly Radisson)
1,56,Thompson,CA,71.76,scattered clouds,55.7435,-97.8558,Burntwood Hotel
2,136,Whitecourt,CA,70.84,broken clouds,54.1502,-115.6855,Super 8 by Wyndham Whitecourt
3,368,High Level,CA,71.71,broken clouds,58.5169,-117.1360,Days Inn by Wyndham High Level
4,127,Atikokan,CA,72.14,broken clouds,48.7555,-91.6216,Quetico Inn (formerly Radisson)


In [13]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
stops_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map.
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(55.0, -100.0), zoom_level=4.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=stops_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))